## Regression and statistics
In the EDA file, I found some correlations between variables. 
Here, I'm testing the variables vs. the outcome - alot of patients (y/n parameter that was created based on the data) 


In [2]:
data <- read.csv('C:/Users/galie/DataScience/project/final_data_inc_newstuff.csv') 
head (data)

,year,month,cum_week1,wk_atm_p,wk_dew_p,wk_avg_max_tmp,wk_max_tm,wk_avg_tm,wk_min_tm,wk_avg_min_tmp,...,max_y_q_precipitation,q_temp,mean_t,mean_t_d,percipitation,max_y_q_temp,min_y_q_temp,max_y_precipitation,seasonality,per_seasonality
,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,2013,1,1,943.7298,18.03250,30.78000,35.7,24.19155,15.8,19.00000,...,84.49714,22.56830,22.5683,17.93779,84.11238,22.5683,17.93779,42.20571,0.16489076,-0.2488066
2,2013,1,2,941.4388,18.37952,28.67143,36.7,22.99917,15.8,19.44286,...,84.49714,22.56830,22.5683,17.93779,84.11238,22.5683,17.93779,42.20571,0.16489076,-0.2488066
3,2013,3,9,943.1124,18.79512,27.91429,36.2,22.44643,16.7,19.18857,...,84.49714,21.43708,22.5683,17.93779,84.11238,22.5683,17.93779,42.20571,-0.01727395,2.4782576
4,2013,9,34,948.7310,13.33714,24.80286,32.0,18.25333,9.2,13.66571,...,84.49714,19.37982,22.5683,17.93779,84.11238,22.5683,17.93779,42.20571,0.38628365,-1.1052278
5,2013,3,7,942.2117,18.41726,30.44286,35.6,23.95131,15.5,18.72000,...,84.49714,21.43708,22.5683,17.93779,84.11238,22.5683,17.93779,42.20571,-0.01727395,2.4782576
6,2013,3,8,942.8707,17.91762,30.70286,35.3,23.93357,15.1,18.53429,...,84.49714,21.43708,22.5683,17.93779,84.11238,22.5683,17.93779,42.20571,-0.01727395,2.4782576


In [1]:
# The 'mechkar' function didn't work, so I found another solution online.

t_table <- function(data, dvs, iv,
                    var_equal = TRUE,
                    p_adj = "none",
                    alpha = 0.05,
                    paired = FALSE,
                    wilcoxon = FALSE) {
  if (!inherits(data, "data.frame")) {
    stop("data must be a data.frame")
  }

  if (!all(c(dvs, iv) %in% names(data))) {
    stop("at least one column given in dvs and iv are not in the data")
  }

  if (!all(sapply(data[, dvs], is.numeric))) {
    stop("all dvs must be numeric")
  }

  if (length(unique(na.omit(data[[iv]]))) != 2) {
    stop("independent variable must only have two unique values")
  }

  out <- lapply(dvs, function(x) {
    if (paired == FALSE & wilcoxon == FALSE) {
      tres <- t.test(data[[x]] ~ data[[iv]], var.equal = var_equal)
    }

    else if (paired == FALSE & wilcoxon == TRUE) {
      tres <- wilcox.test(data[[x]] ~ data[[iv]])
    }

    else if (paired == TRUE & wilcoxon == FALSE) {
      tres <- t.test(data[[x]] ~ data[[iv]],
        var.equal = var_equal,
        paired = TRUE
      )
    }

    else {
      tres <- wilcox.test(data[[x]] ~ data[[iv]],
        paired = TRUE
      )
    }

    c(
      p_value = tres$p.value
    )
  })

  out <- as.data.frame(do.call(rbind, out))
  out <- cbind(variable = dvs, out)
  names(out) <- gsub("[^0-9A-Za-z_]", "", names(out))

  out$p_value <- ifelse(out$p_value < 0.001,
    "<0.001",
    round(p.adjust(out$p_value, p_adj), 3)
  )
  out$conclusion <- ifelse(out$p_value < alpha,
    paste0("Reject H0 at ", alpha * 100, "%"),
    paste0("Do not reject H0 at ", alpha * 100, "%")
  )

  return(out)
}

In [4]:
ls(data)

[1] "alot_of_patients_yn"      "ann_mean_tmp"            
 [3] "ann_precipitation"        "annual_range"            
 [5] "avg_max_blast_w"          "avg_wind_speed"          
 [7] "cum_week1"                "diurnal_range"           
 [9] "dry_week_yn"              "Isothermality"           
[11] "log_num_patients"         "log_wk_avg_precipitation"
[13] "male_perc"                "max_blast_w"             
[15] "max_y_precipitation"      "max_y_q_precipitation"   
[17] "max_y_q_temp"             "max_y_temp"              
[19] "mean_diu_range"           "mean_t"                  
[21] "mean_t_d"                 "mean_temp"               
[23] "min_y_precipitation"      "min_y_q_precipitation"   
[25] "min_y_q_temp"             "min_y_temp"              
[27] "month"                    "monthly_precipitation"   
[29] "num_patients"             "per_seasonality"         
[31] "percipitation"            "q_precipitation"         
[33] "q_temp"                   "season.x"                
[35] "seasonality"              "wk_atm_p"                
[37] "wk_avg_hum"               "wk_avg_max_tmp"          
[39] "wk_avg_min_tmp"           "wk_avg_precipitation"    
[41] "wk_avg_tm"                "wk_dew_p"                
[43] "wk_max_tm"                "wk_min_hum"              
[45] "wk_min_tm"                "year"

In [15]:
# "table 1" - for this, I created the y/n variable since it's only good for categories

result <- t_table(
  data = data,
  c("ann_mean_tmp","ann_precipitation","annual_range","avg_max_blast_w","avg_wind_speed","diurnal_range","dry_week_yn",
    "Isothermality","log_num_patients","log_wk_avg_precipitation","male_perc","max_blast_w","max_y_precipitation",
    "max_y_q_precipitation","max_y_q_temp","max_y_temp","mean_diu_range","mean_t","mean_t_d","mean_temp","min_y_precipitation",
    "min_y_q_precipitation","min_y_q_temp","min_y_temp","monthly_precipitation","per_seasonality","percipitation",
    "q_precipitation","q_temp","season.x","seasonality","wk_atm_p","wk_avg_hum","wk_avg_max_tmp","wk_avg_min_tmp",
    "wk_avg_precipitation","wk_avg_tm","wk_dew_p","wk_max_tm","wk_min_hum","wk_min_tm"),
  "alot_of_patients_yn")

result


variable,p_value,conclusion
<chr>,<chr>,<chr>
ann_mean_tmp,0.016,Reject H0 at 5%
ann_precipitation,<0.001,Reject H0 at 5%
annual_range,<0.001,Reject H0 at 5%
avg_max_blast_w,0.811,Do not reject H0 at 5%
avg_wind_speed,0.281,Do not reject H0 at 5%
diurnal_range,0.002,Reject H0 at 5%
dry_week_yn,<0.001,Reject H0 at 5%
Isothermality,0.03,Reject H0 at 5%
log_num_patients,<0.001,Reject H0 at 5%


In [16]:
library(dplyr)
result %>% arrange (p_value)

variable,p_value,conclusion
<chr>,<chr>,<chr>
ann_precipitation,<0.001,Reject H0 at 5%
annual_range,<0.001,Reject H0 at 5%
dry_week_yn,<0.001,Reject H0 at 5%
log_num_patients,<0.001,Reject H0 at 5%
log_wk_avg_precipitation,<0.001,Reject H0 at 5%
max_y_precipitation,<0.001,Reject H0 at 5%
mean_diu_range,<0.001,Reject H0 at 5%
mean_temp,<0.001,Reject H0 at 5%
min_y_temp,<0.001,Reject H0 at 5%


### most parameters are significant.  

### A linear model for the log_number of patients

In [17]:
mod <- lm(log_num_patients ~ .,data=data)

In [18]:
summary(mod)


Call:
lm(formula = log_num_patients ~ ., data = data)

Residuals:
     Min       1Q   Median       3Q      Max 
-1.38736 -0.36511  0.01326  0.37000  1.54075 

Coefficients: (9 not defined because of singularities)
                           Estimate Std. Error t value Pr(>|t|)    
(Intercept)               3.960e+01  8.496e+02   0.047  0.96286    
year                     -3.455e-02  4.220e-01  -0.082  0.93482    
month                    -1.222e-03  3.932e-02  -0.031  0.97523    
cum_week1                -6.935e-03  8.016e-03  -0.865  0.38769    
wk_atm_p                 -7.084e-03  1.378e-02  -0.514  0.60766    
wk_dew_p                 -2.592e-01  1.389e-01  -1.866  0.06310 .  
wk_avg_max_tmp            6.884e-02  9.573e-02   0.719  0.47268    
wk_max_tm                 2.164e-02  2.811e-02   0.770  0.44217    
wk_avg_tm                -4.090e-02  1.901e-01  -0.215  0.82979    
wk_min_tm                 3.683e-02  3.420e-02   1.077  0.28237    
wk_avg_min_tmp            1.749e-01  

### Multivariate analysis using the num_patients

In [22]:
mod9 <- glm(num_patients ~ ., data=data, family = "poisson")
summary(mod9)


Call:
glm(formula = num_patients ~ ., family = "poisson", data = data)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-0.63592  -0.12356  -0.03184   0.09672   0.35915  

Coefficients: (9 not defined because of singularities)
                           Estimate Std. Error z value Pr(>|z|)    
(Intercept)               6.776e+01  2.315e+02   0.293    0.770    
year                     -3.324e-02  1.151e-01  -0.289    0.773    
month                    -8.935e-04  1.037e-02  -0.086    0.931    
cum_week1                 4.121e-04  2.150e-03   0.192    0.848    
wk_atm_p                 -1.543e-03  3.378e-03  -0.457    0.648    
wk_dew_p                  5.901e-03  4.645e-02   0.127    0.899    
wk_avg_max_tmp            3.502e-03  1.929e-02   0.182    0.856    
wk_max_tm                 7.403e-06  5.981e-03   0.001    0.999    
wk_avg_tm                -8.924e-03  4.699e-02  -0.190    0.849    
wk_min_tm                 1.043e-03  8.554e-03   0.122    0.903    


#### In the previous file, I performed comparisons of the models and found that only:
monthly_precipitation,q_precipitation,q_temp,percipitation,wk_avg_hum, wk_min_hum,wk_dew_p,wk_atm_p had one or 2 hits..

In [23]:
# Since there aren't that many variables anyway, at this point I will remove the non-significant at all variables 
# (i.e. those that were not significant even in the univariate analysis)
# i will remove the year, month and season also, as they correlate too much with the other parameters

library(DataExplorer)
data <- drop_columns(data, c("male_perc","max_y_q_precipitation","min_y_precipitation","max_blast_w","avg_wind_speed",
                             "max_y_q_temp","mean_t","wk_max_tm","avg_max_blast_w","year","month","season.x","diurnal_range"))


In [24]:
head (data)

,cum_week1,wk_atm_p,wk_dew_p,wk_avg_max_tmp,wk_avg_tm,wk_min_tm,wk_avg_min_tmp,wk_avg_hum,wk_min_hum,wk_avg_precipitation,...,monthly_precipitation,q_precipitation,min_y_q_precipitation,q_temp,mean_t_d,percipitation,min_y_q_temp,max_y_precipitation,seasonality,per_seasonality
,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,943.7298,18.03250,30.78000,24.19155,15.8,19.00000,71.23571,25,3.22285714,...,12.822857,84.49714,17.98552,22.56830,17.93779,84.11238,17.93779,42.20571,0.16489076,-0.2488066
2,2,941.4388,18.37952,28.67143,22.99917,15.8,19.44286,77.61310,29,9.60000000,...,12.822857,84.49714,17.98552,22.56830,17.93779,84.11238,17.93779,42.20571,0.16489076,-0.2488066
3,9,943.1124,18.79512,27.91429,22.44643,16.7,19.18857,81.58214,33,12.01714286,...,42.205714,74.85238,17.98552,21.43708,17.93779,84.11238,17.93779,42.20571,-0.01727395,2.4782576
4,34,948.7310,13.33714,24.80286,18.25333,9.2,13.66571,75.72381,23,0.62285714,...,5.681905,29.65905,17.98552,19.37982,17.93779,84.11238,17.93779,42.20571,0.38628365,-1.1052278
5,7,942.2117,18.41726,30.44286,23.95131,15.5,18.72000,73.48571,37,0.02857143,...,42.205714,74.85238,17.98552,21.43708,17.93779,84.11238,17.93779,42.20571,-0.01727395,2.4782576
6,8,942.8707,17.91762,30.70286,23.93357,15.1,18.53429,71.73810,31,0.03428571,...,42.205714,74.85238,17.98552,21.43708,17.93779,84.11238,17.93779,42.20571,-0.01727395,2.4782576


In [25]:
# this is the final data that will be taken to the next stage. 
# writing csv to move to the next notebook.
write.csv(data, "final_final_data.csv", row.names = F) # :)